In [82]:
import sys
import os
import rasterio
import osmnx

import geopandas as gpd
import pandas as pd
import multiprocessing

from shapely.ops import transform
import pyproj

sys.path.append("/home/wb411133/Code/GOSTrocks/src")

import GOSTrocks.ntlMisc as ntlMisc
import GOSTrocks.rasterMisc as rMisc
from GOSTrocks.misc import tPrint

In [53]:
data_folder = "s3://wbg-geography01/URBANIZATION/MENA/"
ucdb_file       = os.path.join(data_folder,"Extents", "GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg")
fua_file        = os.path.join(data_folder,"Extents", "GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg")
fua_peripheries = os.path.join(data_folder,"Extents", "FUA_peripheries.gpkg")

# Highway features are reclassified to 4 OSMLR classes for simplification and standardization
#   https://mapzen.com/blog/osmlr-2nd-technical-preview/
OSMLR_Classes = {
"motorway":"OSMLR_1",
"motorway_link":"OSMLR_1",
"trunk":"OSMLR_1",
"trunk_link":"OSMLR_1",
"primary":"OSMLR_1",
"primary_link":"OSMLR_1",

"secondary":"OSMLR_2",
"secondary_link":"OSMLR_2",
"tertiary":"OSMLR_2",
"tertiary_link":"OSMLR_2",

"unclassified":"OSMLR_3",
"unclassified_link": "OSMLR_3",
"residential": "OSMLR_3",
"residential_link": "OSMLR_3",

"track": "OSMLR_4",
"service": "OSMLR_4"
}


In [46]:
inF = gpd.read_file(fua_file)
inF = inF.to_crs(4326)
inP = gpd.read_file(fua_peripheries)
if inP.crs != inF.crs:
    inP = inP.to_crs(4326)
inP['geometry'] = inP.buffer(0)

TypeError: 'method' object is not subscriptable

In [52]:
wgs84 = pyproj.CRS('EPSG:4326')
wm = pyproj.CRS('EPSG:3278')

project = pyproj.Transformer.from_crs(wgs84, wm, always_xy=True).transform

def summarize_road_length(curR, core_shp, fua_shp, transformer):
    ''' curR - GeoDataFrame of roads
    core_shp - shapely polygon of core
    transformer - turn wgs84 into 3857
    
    returns - array of length intersecting [core_shp, fua_shp] in metres
    '''
    core_rds = curR.unary_union.intersection(core_shp)
    core_rds = transform(transformer, core_rds)
    fua_rds = curR.unary_union.intersection(fua_shp)
    fua_rds = transform(transformer, fua_rds)    
    return([core_rds.length, fua_rds.length])

In [58]:
all_res = []
for idx, cur_city in inF.iterrows():
    cur_roads = osmnx.geometries_from_polygon(cur_city['geometry'], {"highway":True})
    cur_roads['OSMLR'] = cur_roads['highway'].map(OSMLR_Classes)
    cur_periphery = inP.loc[inP['eFUA_ID'] == cur_city['eFUA_ID']].iloc[0]
    cur_res = {"eFUA_ID":cur_city['eFUA_ID']}
    for lbl, rds in cur_roads.groupby('OSMLR'):
        rd_length = summarize_road_length(rds, cur_city.geometry, cur_periphery.geometry, project)
        cur_res[f'core_{lbl}_m'] = rd_length[0]
        cur_res[f'per_{lbl}_m'] = rd_length[1]
    all_res.append(cur_res)
    tPrint(idx)

11:41:51	0
11:42:53	1
11:43:54	2
11:44:55	3
11:45:57	4
11:46:59	5


KeyboardInterrupt: 

# Multiprocessing

In [100]:
def summarize_road_length(cur_city, cur_p):
    ''' 
    cur_city - geoseries of current city attributes
    cur_p - geoseries of matching periphery
    transformer - turn wgs84 into 3857
    
    returns - array of length intersecting [e_ID, core_shp, fua_shp] in metres
    '''
    wgs84 = pyproj.CRS('EPSG:4326')
    wm = pyproj.CRS('EPSG:3278')
    transformer = pyproj.Transformer.from_crs(wgs84, wm, always_xy=True).transform
    cur_res = {'eFUA_ID':cur_city['eFUA_ID']}
    try:
        try:
            cur_roads = osmnx.geometries_from_polygon(cur_city['geometry'], {"highway":True})
        except:
            cur_roads = osmnx.geometries_from_polygon(cur_city['geometry'].buffer(0), {"highway":True})
        cur_roads['OSMLR'] = cur_roads['highway'].map(OSMLR_Classes)
        for lbl, curR in cur_roads.groupby('OSMLR'):
            core_rds = curR.unary_union.intersection(cur_city.geometry)
            core_rds = transform(transformer, core_rds)
            fua_rds = curR.unary_union.intersection(cur_p.geometry)
            fua_rds = transform(transformer, fua_rds)
            cur_res[f'core_{lbl}_m'] = core_rds.length
            cur_res[f'per_{lbl}_m'] = fua_rds.length
        tPrint(f"Completed {cur_city['eFUA_name']}")
    except:
        tPrint(f"*****Error {cur_city['eFUA_name']}")
    return(cur_res)


/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [101]:
summarize_road_length(*all_args[0])

02:15:13	Completed Herat


{'eFUA_ID': 1.0,
 'core_OSMLR_1_m': 312986.3572159438,
 'per_OSMLR_1_m': 51388.56508881661,
 'core_OSMLR_2_m': 781475.624358757,
 'per_OSMLR_2_m': 308737.8013207163,
 'core_OSMLR_3_m': 4028535.265775058,
 'per_OSMLR_3_m': 1562502.6308872837,
 'core_OSMLR_4_m': 433131.6598219003,
 'per_OSMLR_4_m': 265436.7277154675}

In [84]:
# prepare mp arguments
all_args = []
for idx, cur_city in inF.iterrows():
    cur_periphery = inP.loc[inP['eFUA_ID'] == cur_city['eFUA_ID']].iloc[0]
    all_args.append([cur_city, cur_periphery])

In [102]:
with multiprocessing.Pool(70) as pool:
    res = pool.starmap(summarize_road_length, all_args)

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


02:15:19	Completed Tamacine
02:15:19	Completed Ksar El Boukhari
02:15:19	Completed M'banza Congo
02:15:20	*****Error Bayuquan
02:15:20	*****Error Nanning
02:15:20	Completed Shamakhi
02:15:20	*****Error Guangzhou
02:15:20	Completed Yi'an
02:15:20	Completed Elbasan
02:15:20	Completed Bambari
02:15:20	Completed Chelghoum Laid
02:15:20	Completed Gaoua
02:15:20	Completed Ngozi
02:15:20	Completed Daozhen
02:15:21	Completed Yangxian
02:15:21	Completed Foumban
02:15:21	Completed Gabela
02:15:21	Completed Quxian
02:15:21	Completed Jinchang
02:15:21	Completed Dajie
02:15:21	Completed Danfeng
02:15:21	Completed Shouyang
02:15:21	Completed Hengche
02:15:21	Completed Yeji
02:15:21	Completed Saatli
02:15:21	Completed Hulunbuir
02:15:21	Completed Antu
02:15:21	Completed Barkhada
02:15:21	Completed Luotian
02:15:21	Completed Paranaguá
02:15:21	Completed Yunyang
02:15:21	Completed Janaúba
02:15:21	Completed Dahua
02:15:21	Completed Gongqingcheng02:15:21	Completed Yuanling

02:15:21	Completed Bagé
02:15

02:15:45	Completed Xiji02:15:45	Completed Yacuiba

02:15:45	Completed Zalantun
02:15:45	Completed Huairou
02:15:45	*****Error Changzhou
02:15:45	Completed Weinan
02:15:45	Completed Qingyuan
02:15:45	Completed Chuzhou
02:15:45	Completed Li
02:15:46	Completed Katchiungo
02:15:46	Completed Jinzhong
02:15:46	Completed Zhongyang
02:15:46	Completed Gore
02:15:46	Completed Shuangfeng
02:15:46	Completed Yushe
02:15:46	Completed Ji'an
02:15:46	Completed Jining
02:15:46	Completed Kunshe
02:15:46	Completed La Serena
02:15:46	Completed Benxi
02:15:46	Completed Carazinho
02:15:46	Completed 莒南县
02:15:46	Completed Shayuan
02:15:46	Completed Yanchi
02:15:46	Completed Dangyang
02:15:46	Completed Dourados
02:15:46	Completed Zuoyun
02:15:46	Completed UNNAMED
02:15:46	Completed Weishi
02:15:46	Completed Changzhi
02:15:46	Completed Ankang
02:15:47	Completed Xinxing
02:15:47	Completed Shihezi
02:15:47	Completed Garoua
02:15:47	Completed Lechang
02:15:47	Completed Danjiangkou
02:15:47	Completed Jiajiang
02:1

02:16:13	Completed Libertador Gral. San Martín
02:16:13	Completed Yuxi
02:16:13	Completed Xiangshui
02:16:13	Completed Jianghua
02:16:13	Completed San Antonio
02:16:13	Completed Nagqu
02:16:13	Completed 三原县
02:16:13	Completed 宁阳县02:16:13	Completed Xunwu

02:16:13	Completed 中江县
02:16:14	*****Error Shangqiu
02:16:14	Completed Hefei
02:16:14	Completed Tongling
02:16:14	Completed Bagram
02:16:14	Completed Xichong
02:16:14	Completed Djelfa
02:16:14	Completed Cabinda
02:16:14	Completed Pelotas
02:16:14	Completed Pingyuan
02:16:14	Completed Hanchuan
02:16:14	Completed Siping
02:16:14	Completed Longyan
02:16:14	Completed Baoying
02:16:14	Completed Pingchuan
02:16:14	Completed Pergamino
02:16:14	Completed Djibo
02:16:14	Completed Liangmatai
02:16:14	Completed Dongtai
02:16:14	Completed Umuarama
02:16:14	*****Error Nanchang
02:16:14	Completed Hechuan
02:16:14	Completed Dianjiang
02:16:14	Completed Qianxi
02:16:14	Completed Relizane
02:16:14	Completed Dazhu
02:16:14	Completed Jiagedaqi
02:16:14	C

02:16:46	Completed Sheqi
02:16:46	Completed Gonder
02:16:46	Completed Bumba
02:16:46	Completed Lubutu
02:16:47	Completed Rubaya
02:16:47	Completed Wuping02:16:47	Completed Mekane Yesus

02:16:47	Completed The State Of Tigray
02:16:47	Completed Brahmanbaria
02:16:47	Completed Arbe Gona
02:16:47	Completed Dera
02:16:47	Completed Concepción del Uruguay
02:16:47	Completed Tulqa
02:16:47	Completed Shifang
02:16:47	Completed Cacoal
02:16:47	Completed Agaro
02:16:47	Completed Qaryat Salah al Din
02:16:47	Completed Robe
02:16:47	Completed Gedeb
02:16:47	Completed Kasongo
02:16:47	Completed Fuqing
02:16:48	Completed Sandoa
02:16:48	Completed Weldiya
02:16:48	Completed Shahura (Alefa)
02:16:48	Completed Sodo
02:16:48	Completed Rio Verde
02:16:48	Completed Manacapuru
02:16:48	Completed Kamanyola
02:16:48	Completed UNNAMED
02:16:48	Completed Meki
02:16:48	Completed Bahir Dar
02:16:48	Completed Geziret Nikla
02:16:48	Completed Ningguo
02:16:48	Completed Malemba Nkulu
02:16:48	Completed Tshimbulu
02

...

02:28:33	Completed Jelenia Góra
02:28:33	Completed Orenburg
02:28:34	Completed Chincha Alta
02:28:34	Completed Kirov
02:28:35	Completed Alwar
02:28:35	Completed Novouralsk
02:28:35	Completed Târgu Mureș
02:28:37	Completed Almaty
02:28:37	Completed Udine
02:28:37	Completed Novocheboksarsk
02:28:37	Completed Ryazan
02:28:37	Completed Lodz
02:28:37	Completed Slatina
02:28:38	Completed Xanthi
02:28:38	Completed Monterrey
02:28:38	Completed Al Arish
02:28:38	Completed Huancayo
02:28:38	Completed Lubin
02:28:39	Completed Labinsk
02:28:39	Completed Yelets
02:28:39	Completed Samara
02:28:40	Completed Pedro Juan Caballero
02:28:40	Completed Ciudad Mante
02:28:40	Completed Tarakai
02:28:41	Completed Pucallpa
02:28:41	Completed Pitești
02:28:41	Completed Kovrov
02:28:41	Completed Divinópolis
02:28:42	Completed Bugama
02:28:42	Completed Musarfirkhana
02:28:42	Completed Pichanaki
02:28:43	Completed Volgodonsk
02:28:43	Completed Kineshma
02:28:44	Completed Ayacucho
02:28:45	Completed Legnica
02:28:4

02:32:54	Completed Poyang
02:32:54	Completed Lixin
02:32:55	Completed Pará de Minas
02:32:55	Completed Khashm El Girba
02:32:55	Completed UNNAMED
02:32:56	Completed Murcia
02:32:56	Completed Aurangabad
02:32:56	Completed Nyala02:32:56	Completed Sonkach

02:32:57	Completed Buram
02:32:57	Completed UNNAMED
02:32:58	Completed Oyama
02:32:58	Completed Ourense
02:32:58	Completed Kassala
02:32:58	Completed Sinkāt
02:32:59	Completed Trapani
02:32:59	Completed Pahasu
02:33:00	Completed Al-Fashir
02:33:00	Completed Jablah
02:33:00	Completed Gijón
02:33:00	Completed Žilina
02:33:01	Completed Mellit
02:33:01	Completed 惠来县
02:33:01	Completed Baniyas
02:33:01	Completed Araruama
02:33:01	Completed UNNAMED
02:33:01	Completed Tomakomai
02:33:01	Completed Ceuta
02:33:02	Completed Huelva
02:33:02	Completed UNNAMED
02:33:02	Completed Port Sudan
02:33:02	Completed Cosenza
02:33:02	Completed Chuncheon-si
02:33:02	Completed Tartus
02:33:03	Completed Sawakin
02:33:03	Completed Malacatán
02:33:03	Completed Mt

02:36:22	Completed Zhytomyr
02:36:23	Completed Bari
02:36:24	Completed Berdychiv
02:36:24	Completed Hesperia
02:36:25	Completed Kapaklı
02:36:25	Completed Hakodate
02:36:25	Completed Geneva
02:36:27	Completed Tosham
02:36:28	Completed Wonju-si
02:36:28	Completed Manresa
02:36:28	Completed Mahilyow
02:36:29	Completed Luanzhou
02:36:29	Completed Yuba City
02:36:31	Completed Seeb
02:36:32	Completed Mengzhou
02:36:32	Completed Kohat
02:36:32	Completed Vinnytsia
02:36:32	*****Error Kyiv
02:36:34	Completed McAllen
02:36:35	Completed Zhangjiajie
02:36:36	Completed Luoyuan
02:36:37	Completed Chamarajanagara
02:36:38	Completed Semarang
02:36:38	Completed Lviv
02:36:40	Completed Guangrao
02:36:40	Completed Shicheng
02:36:40	*****Error Zhangjiagang
02:36:41	Completed Stryi
02:36:42	Completed Ile Oluji
02:36:43	Completed Chervonohrad
02:36:43	Completed Lumding
02:36:43	Completed Jinghai
02:36:43	Completed Kharkiv
02:36:44	Completed Kamensk-Shakhtinsky
02:36:45	Completed Kovel
02:36:47	Completed Ja

02:41:22	Completed Pejë
02:41:22	Completed Mohnyin
02:41:22	Completed Yunhe
02:41:24	Completed Kitami
02:41:26	Completed Joetsu
02:41:26	Completed Syktyvkar
02:41:26	Completed Aguacatán
02:41:27	Completed New Taipei [Taipei]
02:41:27	Completed Nibua Raiganj
02:41:27	Completed Tengzhou
02:41:29	Completed Chirchiq
02:41:31	Completed Biloxi
02:41:33	Completed Bozhou
02:41:33	Completed Graz
02:41:35	Completed Tabuk
02:41:36	Completed Thọ Xuân
02:41:36	Completed Hikone
02:41:36	*****Error Nagoya
02:41:36	Completed Kapasan
02:41:40	Completed Leeuwarden
02:41:42	Completed Białystok
02:41:42	Completed Nashville
02:41:45	Completed Fengfeng
02:41:46	Completed Turgutlu
02:41:48	Completed Dortmund
02:41:49	Completed Mithapur
02:41:49	Completed Mandeville
02:41:49	Completed Gwadar
02:41:50	Completed Aqaba
02:41:50	Completed Wani
02:41:50	*****Error Hyderabad
02:41:51	Completed Colorado Springs
02:41:51	Completed Mahuta
02:41:52	Completed 瓮安县
02:41:56	Completed Pervomaisk
02:41:58	Completed Zelenogr

02:59:12	Completed Al Khor
02:59:20	Completed Warwick
02:59:25	Completed Folkestone
02:59:31	Completed Bern
02:59:35	Completed Chester
02:59:41	Completed Christchurch
02:59:47	Completed Leuven
02:59:49	Completed Richmond
02:59:50	Completed Kamianske
02:59:50	Completed İzmit
02:59:58	Completed Hartford
03:00:01	Completed Al Beidha
03:00:01	Completed Ulm
03:00:05	Completed Erie
03:00:05	*****Error Virginia Beach
03:00:05	*****Error Washington D.C.
03:00:08	Completed Milwaukee
03:00:12	Completed Changwon
03:00:17	Completed Quảng Ngãi
03:00:19	Completed Pointe-Noire
03:00:23	Completed Racine
03:00:23	Completed Bielefeld
03:00:38	Completed Houston
03:00:43	Completed Carlisle
03:00:44	Completed Coventry
03:00:47	Completed College Station
03:00:57	Completed Denizli
03:01:01	Completed Wellington
03:01:02	Completed Lawdar
03:01:11	Completed Basel
03:01:16	Completed Gyeongju-si
03:01:16	*****Error Busan
03:01:16	*****Error Geoje-si
03:01:16	Completed Norwich
03:01:18	Completed Quảng Phú
03:01:21

In [135]:
osm_res = pd.DataFrame(res)
bad_fua = osm_res.loc[osm_res['core_OSMLR_2_m'].isna() * osm_res['core_OSMLR_3_m'].isna(),'eFUA_ID'].values
updatedF = inF.copy()
updatedF.set_index("eFUA_ID")
all_res = []
for cur_fua in bad_fua:
    cur_city = inF.loc[inF['eFUA_ID'] == cur_fua].iloc[0]
    cur_periphery = inP.loc[inP['eFUA_ID'] == cur_city['eFUA_ID']].iloc[0]
    try:
        cur_city['geometry'] = cur_city['geometry'].buffer(0)
        cur_periphery['geometry'] = cur_periphery['geometry'].buffer(0)
        cur_res = summarize_road_length(cur_city, cur_periphery)
        all_res.append(cur_res)
    except:
        tPrint("")

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(


09:09:31	Completed Brisbane
09:09:48	Completed Manama
09:10:00	Completed Comilla
09:10:08	Completed Chattogram
09:10:17	Completed Santos
09:10:21	Completed Taubaté
09:11:20	Completed Rio de Janeiro
09:11:23	Completed Ipatinga
09:11:23	
09:25:10	Completed Montreal
09:26:20	Completed Trois-Rivières
09:26:20	
09:27:36	Completed Chongqing
09:28:45	Completed Luoyang
09:29:48	Completed Renqiu
09:30:52	Completed Jiaozuo
09:31:55	Completed Gongyi
09:33:05	Completed Xinxiang
09:34:21	Completed Zhengzhou
09:35:27	Completed Binhai
09:36:31	Completed Gucheng
09:37:34	Completed Qinhuangdao
09:38:37	Completed Jinzhou
09:39:41	Completed Shangqiu
09:40:45	Completed 兖州区
09:44:56	Completed Zibo
09:46:04	Completed Nanning
09:47:07	Completed Bayuquan
09:48:15	Completed Weifang
09:52:37	Completed Wuhan
09:53:48	Completed Linyi
09:54:52	Completed Laiyang
09:59:08	Completed Qingdao
10:00:13	Completed Huangdao
10:01:22	Completed Nanchang
10:08:16	Completed Guangzhou
10:09:20	Completed Guangzhou
10:10:25	Compl

In [145]:
fixed_roads

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,core_OSMLR_1_m,per_OSMLR_1_m,core_OSMLR_2_m,per_OSMLR_2_m,core_OSMLR_3_m,per_OSMLR_3_m,core_OSMLR_4_m,per_OSMLR_4_m
eFUA_ID,,,,,,,,
2355.0,2.929011e+06,1.653525e+06,4.157088e+06,2.442701e+06,1.423827e+07,8.218409e+06,7.149086e+06,4.936160e+06
16.0,1.898748e+06,4.787725e+05,3.155987e+06,7.561082e+05,1.499426e+07,3.254439e+06,8.151493e+06,3.097810e+06
4316.0,1.690944e+06,4.743068e+05,3.834489e+06,1.156027e+06,2.536710e+07,7.847487e+06,5.185805e+05,1.497565e+05
4833.0,1.476681e+06,2.706563e+05,1.100881e+06,1.945202e+05,1.236171e+07,3.487265e+06,1.242897e+06,3.560951e+05
6700.0,6.280952e+05,2.001665e+05,1.050265e+06,1.390389e+05,5.570813e+06,1.725545e+06,6.468513e+05,1.912925e+05
...,...,...,...,...,...,...,...,...
2481.0,6.158593e+06,1.336972e+06,6.471010e+06,2.047333e+06,1.102023e+07,5.515415e+06,1.939923e+07,5.580180e+06
3428.0,1.198926e+06,3.017537e+05,1.593776e+06,5.338213e+05,9.682477e+06,3.293933e+06,2.597407e+06,1.081946e+06
2207.0,7.510771e+06,2.622645e+06,1.000996e+07,3.657831e+06,6.165523e+07,2.776612e+07,1.873443e+07,4.891911e+06


In [143]:
fixed_roads = pd.DataFrame(all_res)
fixed_roads = fixed_roads.set_index('eFUA_ID')
final_dataset = osm_res.set_index('eFUA_ID')
final_dataset.update(fixed_roads)

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [147]:
final_dataset.to_csv(os.path.join(data_folder, "ZONAL_RES", "OSM_ROAD_LENGTH", "OSMLR_Classes_length_m.csv"))

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/pandas/compat/_optional.py:116: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(version) < minimum_version:
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/setuptools/_distutils/version.py:345: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/wb411133/.conda/envs/ee/lib/python3.9/asyncio/sslproto.py:320: ResourceWarning: unclosed transport <asyncio.sslproto._SSLProtocolTransport object at 0x7f11e88e2ca0>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)
